<h2>Test/Val 1 TentaCar</h2>

In [ ]:
import sim
import numpy as np
import math
import time
import cv2
import matplotlib.pyplot as plt
!pip install tflite-runtime opencv-python numpy
from tensorflow.lite.python.interpreter import Interpreter

### 1. Establecer la conexión
Utilizaremos las funciones del API Remoto de COPPELIA.


In [ ]:
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [ ]:
# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
sim.simxFinish(-1)
client_id = connect(19999)

<h1> Clase de Visionado y identificación de objetos</h1>


In [ ]:
class VisionModuleSim:
    def __init__(self, client_id, model_path='efficientdet_lite0.tflite'):
        self.client_id = client_id

        # Obtener handles       
        ret, self.cam_handle = sim.simxGetObjectHandle(self.client_id, 'Prox', sim.simx_opmode_blocking)
        ret, self.sensor_handle = sim.simxGetObjectHandle(self.client_id, 'Vision_sensor', sim.simx_opmode_blocking)
        
        # Inicializar streaming correctamente:
        # Para sensor de visión (imagen):
        sim.simxGetVisionSensorImage(self.client_id, self.sensor_handle, 0, sim.simx_opmode_streaming)
        # Para sensor de proximidad (distancia):
        sim.simxReadProximitySensor(self.client_id, self.cam_handle, sim.simx_opmode_streaming)
        time.sleep(1)

        # Cargar modelo TFLite
        self.interpreter = Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        input_details = self.interpreter.get_input_details()
        self.input_index = input_details[0]['index']
        self.input_shape = input_details[0]['shape']

        output_details = self.interpreter.get_output_details()
        self.output_indices = {
            'boxes': output_details[0]['index'],
            'classes': output_details[1]['index'],
            'scores': output_details[2]['index'],
            'num_detections': output_details[3]['index'],
        }

    def preprocess_image(self, image):
        img = cv2.resize(image, (self.input_shape[2], self.input_shape[1]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.uint8)  # para modelo uint8
        img = np.expand_dims(img, axis=0)
        return img

    def get_detections(self):
        # Obtener imagen de la cámara simulada (sensor de visión)
        res, resolution, image = sim.simxGetVisionSensorImage(self.client_id, self.sensor_handle, 0, sim.simx_opmode_oneshot_wait)
        if res != sim.simx_return_ok:
            print(f"Error al obtener imagen del sensor de visión: {res}")
            return []

        img=np.array(image,dtype=np.float32)
        img = (1.0 - img) * 255
        img = img.astype(np.uint8)
        img= img.reshape((resolution[1],resolution[0],3))
        img = cv2.flip(img, 0)  # Voltear verticalmente

        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title("Imagen recibida del sensor")
        plt.show()

        input_tensor = self.preprocess_image(img)
        self.interpreter.set_tensor(self.input_index, input_tensor)
        self.interpreter.invoke()

        boxes = self.interpreter.get_tensor(self.output_indices['boxes'])[0]  # [N,4]
        classes = self.interpreter.get_tensor(self.output_indices['classes'])[0]  # [N]
        scores = self.interpreter.get_tensor(self.output_indices['scores'])[0]  # [N]
        num = int(self.interpreter.get_tensor(self.output_indices['num_detections'])[0])

        detections = []
        for i in range(num):
            if scores[i] > 0.3:
                ymin, xmin, ymax, xmax = boxes[i]
                detections.append({
                    'class_id': int(classes[i]),
                    'score': scores[i],
                    'bbox': (xmin, ymin, xmax, ymax)
                })
        return detections

    def get_nearest_object_distance(self):
        # Leer distancia del sensor de proximidad
        errorCode, detected, distance_data, _, _ = sim.simxReadProximitySensor(self.client_id, self.cam_handle, sim.simx_opmode_blocking)
        print(f"ErrorCode: {errorCode}, Detectado: {detected}, Datos: {distance_data}")
        if detected:
            sensor_val = np.linalg.norm(distance_data)
            return round(sensor_val * 100, 2)  # cm
        else:
            return -1


<h1> Codi Moure Robot</h1>


In [ ]:
class CarSim:
    def __init__(self, client_id):
        self.client_id = client_id
        self.speed = 2  # rad/s, ajustar según el modelo

        # Obtener handles de los motores
        ret,self.right_motor=sim.simxGetObjectHandle(self.client_id,'Revolute_joint_right',sim.simx_opmode_blocking)
        ret,self.left_motor=sim.simxGetObjectHandle(self.client_id,'Revolute_joint_left',sim.simx_opmode_blocking)
        ret,self.back_motor=sim.simxGetObjectHandle(self.client_id,'Revolute_joint_behind',sim.simx_opmode_blocking)

    def move_forward(self, duration=0.2):
        sim.simxSetJointTargetVelocity(self.client_id, self.left_motor, self.speed, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(self.client_id, self.right_motor, self.speed, sim.simx_opmode_streaming)
        time.sleep(duration)
        self.stop()

    def stop(self):
        sim.simxSetJointTargetVelocity(self.client_id, self.left_motor, 0, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(self.client_id, self.right_motor, 0, sim.simx_opmode_streaming)

    def turn_right(self, rotation=1):
        sim.simxSetJointTargetVelocity(self.client_id, self.left_motor, self.speed, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(self.client_id, self.right_motor, -self.speed, sim.simx_opmode_streaming)
        time.sleep(0.2 * rotation)
        self.stop()

    def turn_left(self, rotation=1):
        sim.simxSetJointTargetVelocity(self.client_id, self.left_motor, -self.speed, sim.simx_opmode_streaming)
        sim.simxSetJointTargetVelocity(self.client_id, self.right_motor, self.speed, sim.simx_opmode_streaming)
        time.sleep(0.2 * rotation)
        self.stop()

    def cleanup(self):
        self.stop()
        sim.simxFinish(self.client_id)

<h1>Codi Deteccions</h1>

In [ ]:
# Parámetros de estilo
_MARGIN = 10  # pixeles
_ROW_SIZE = 10
_FONT_SIZE = 1
_FONT_THICKNESS = 1
_TEXT_COLOR = (0, 0, 255)  # rojo

def visualize_detections(image, detections):
    """
    Dibuja bounding boxes y etiquetas en una imagen (detecciones TFLite).

    Args:
        image: Imagen RGB (numpy array)
        detections: lista de dicts con claves: 'class_id', 'score', 'bbox' (xmin, ymin, xmax, ymax) en [0,1]

    Returns:
        Imagen con anotaciones
    """
    h, w, _ = image.shape
    for det in detections:
        xmin, ymin, xmax, ymax = det['bbox']
        xmin = int(xmin * w)
        xmax = int(xmax * w)
        ymin = int(ymin * h)
        ymax = int(ymax * h)

        # Dibuja caja
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), _TEXT_COLOR, 2)

        # Etiqueta
        label = f"{det.get('label', 'obj')} ({round(det['score'], 2)})"
        text_location = (_MARGIN + xmin, _MARGIN + _ROW_SIZE + ymin)
        cv2.putText(image, label, text_location, cv2.FONT_HERSHEY_PLAIN,
                    _FONT_SIZE, _TEXT_COLOR, _FONT_THICKNESS)

    return image

# Para mostrar en Jupyter:
def show_image_in_jupyter(img_rgb):
    plt.figure(figsize=(8, 6))
    plt.imshow(cv2.cvtColor(img_rgb, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

<h1>Codi Algoritme Cotxe</h1>

In [ ]:
def get_orientation(image, detection, distancia_cm, fov_deg=60):
    _, w, _ = image.shape

    xmin, ymin, xmax, ymax = detection['bbox']
    cx_px = ((xmin + xmax) / 2) * w
    mitad_imagen = w / 2
    offset_px = cx_px - mitad_imagen

    # Ángulo por píxel
    angle_per_pixel = fov_deg / w
    orientation_deg = offset_px * angle_per_pixel

    return orientation_deg

    
def calcular_tiempo_rotacion(angulo_deg, velocidad, distancia_entre_ruedas):
    theta_rad = math.radians(angulo_deg)
    omega = 2 * velocidad / distancia_entre_ruedas
    tiempo = abs(theta_rad) / omega
    return tiempo
    
def bug2(client_id, vision_module, car, image, detection):
    distance = vision_module.get_nearest_object_distance()
    orientation = get_orientation(image, detection, distance)
    segundos = calcular_tiempo_rotacion(orientation, 2, 0.03)
    if(orientation > 0):
        car.turn_right(segundos+0.5)
    else:
        car.turn_left(segundos+0.5)
    time.sleep(0.05)
    sim.simxReadProximitySensor(client_id, vision_module.sensor_handle, sim.simx_opmode_streaming)

    while True:
        distance = vision_module.get_nearest_object_distance()
        print(distance)
        if distance < 30 and distance != -1:
            car.stop()
            print("Objetivo alcanzado")
            break

        orientation = get_orientation(image, detection, distance)
        segundos = calcular_tiempo_rotacion(orientation, 2, 0.03)
        if(orientation > 0):
            car.turn_right(segundos + 0.04)
        else:
            car.turn_left(segundos + 0.04)
        time.sleep(0.1)
        car.move_forward()
        time.sleep(0.1)

<h1>Codi Pinça</h1>

In [ ]:
class Pinca:
     def __init__(self, client_id, vision_module):
        self.client_id = client_id
        self.speed = 2  # rad/s, ajustar según el modelo
        self.vision_module = vision_module

        # Obtener handles de los motores
        ret,self.girar_gancho=sim.simxGetObjectHandle(self.client_id,'gancho1',sim.simx_opmode_blocking)
        ret,self.control_gancho=sim.simxGetObjectHandle(self.client_id,'gancho2',sim.simx_opmode_blocking)
        ret,self.pinza1=sim.simxGetObjectHandle(self.client_id,'pinza1',sim.simx_opmode_blocking)
        ret,self.pinza2=sim.simxGetObjectHandle(self.client_id,'pinza2',sim.simx_opmode_blocking) #servo adicional coppelia

     def reset_position(self):
        qr0 = 0 * np.pi/180
        qr1 = 0 * np.pi/180
        qr2 = 0 * np.pi/180
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.girar_gancho, qr0, sim.simx_opmode_oneshot)
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.control_gancho, qr1, sim.simx_opmode_oneshot)
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza1, qr2, sim.simx_opmode_oneshot)
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza2, qr2, sim.simx_opmode_oneshot)
    
     def home_position(self):
        qr0 = 0 * np.pi/180
        qr1 = 0 * np.pi/180
        qr2 = 0 * np.pi/180
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.girar_gancho, qr0, sim.simx_opmode_oneshot)
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.control_gancho, qr1, sim.simx_opmode_oneshot)

     def convert_to_coppelia(self,x,y):
        resolution0 = 1280
        resolution1 = 720
        x_world = (x - resolution0 / 2) / resolution0 * -0.2
        y_world = (y - resolution1 / 2) / resolution1 * -0.2
        return x_world, y_world
        
     def dirigir_a(self, x, y, z):
      
        brazo_fijo = 0.2   # brazo a 45º (fijo)
        brazo_movil = 0.2  # brazo que sube y baja
        h_base = 0.05       # altura base al suelo
    
        # 1. Joint 1: rotación base
        theta1 = math.atan2(y, x)
    
        # 2. Pasar coordenadas al plano vertical inclinado a 45°
        x_horizontal = math.hypot(x, y)
    
        # Compensar brazo fijo (offset)
        dx = brazo_fijo / math.sqrt(2)
        dz = brazo_fijo / math.sqrt(2)
    
        x2 = x_horizontal - dx
        z2 = z - h_base - dz
    
        # 3. Proyectar al plano del brazo (gira a 45°) → como brazo plano 2D
        # En el plano inclinado, transformamos (x2, z2) a coordenadas locales del plano
        # Rotación -45º: sistema del brazo
        x_local = x2 * math.cos(-math.pi/4) + z2 * math.sin(-math.pi/4)
        z_local = -x2 * math.sin(-math.pi/4) + z2 * math.cos(-math.pi/4)
    
        # 4. Ángulo para Joint 2 (cinemática inversa plana 1DOF)
        if z_local > brazo_movil:
            print("¡Punto fuera del alcance!")
            return None
    
        # Consideramos el brazo como girando desde el eje 0 hacia abajo
        theta2 = math.atan2(z_local, x_local)
    
        # Convertir a grados
        theta1_deg = math.degrees(theta1)
        theta2_deg = math.degrees(theta2)
    
        return theta1_deg, theta2_deg

     def gripper(self, open_close):
        if open_close == 1: #abrir
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza1, 30, sim.simx_opmode_blocking)
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza2, -30, sim.simx_opmode_blocking)
        elif open_close == 0: #cerrar
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza1, -2, sim.simx_opmode_blocking)
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza2, 2, sim.simx_opmode_blocking)
        else: #reset
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza1, 0, sim.simx_opmode_blocking)
            returnCode = sim.simxSetJointTargetPosition(self.client_id, self.pinza2, 0, sim.simx_opmode_blocking)


     def agafar_objecte(self):
        detections = self.vision_module.get_detections()
        detection = detections[0]
        _, _, image = sim.simxGetVisionSensorImage(client_id, self.vision_module.sensor_handle, 0, sim.simx_opmode_oneshot_wait)
        img=np.array(image,dtype=np.float32)
        img = (1.0 - img) * 255
        img = img.astype(np.uint8)
        img= img.reshape((resolution[1],resolution[0],3))
        img = cv2.flip(img, -1)
        h , w, _ = img.shape
        xmin, ymin, xmax, ymax = detection['bbox']
        cx_px = ((xmin + xmax) / 2) * w
        cy_px = ((ymin + ymax) / 2) * h
        x, y = self.convert_to_coppelia(cx_px, cy_px)
        print(x, y)
        theta1_deg, theta2_deg = self.dirigir_a(x, y)
        print(theta1_deg, theta2_deg)
        q = [0,0]
        q[0] = theta1_deg*np.pi/180
        q[1] = theta2_deg*np.pi/180
        
        self.gripper(1) #abrir pinza
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.girar_gancho, q[0], sim.simx_opmode_blocking)
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.control_gancho, q[1], sim.simx_opmode_blocking)
        time.sleep(2)
        self.gripper(0) #cerrar pinza
        time.sleep(1)
        self.home_position()
        returnCode = sim.simxSetJointTargetPosition(self.client_id, self.girar_gancho, np.pi, sim.simx_opmode_blocking)
        time.sleep(0.5)
        self.gripper(1) #abrir pinza
        time.sleep(3)
        self.gripper(2) #reset pinza
        self.reset_position()
        
        

        


<h2>Test 1. Validación de la cámara</h2>

In [ ]:
vision_module = VisionModuleSim(client_id)

res, resolution, image = sim.simxGetVisionSensorImage(client_id, vision_module.sensor_handle, 0, sim.simx_opmode_oneshot_wait)
print(f"Lectura imagen res: {res}, resolution: {resolution}")
assert res == sim.simx_return_ok, "❌ No se pudo obtener imagen del sensor"
print("✅ Test 2: Imagen de cámara recibida")

# Mostrar imagen
img=np.array(image,dtype=np.float32)
img = (1.0 - img) * 255
img = img.astype(np.uint8)
img= img.reshape((resolution[1],resolution[0],3))
img = cv2.flip(img, -1)
show_image_in_jupyter(img)


<h2>Test 2. Validación Detección Objetos</h2>

In [ ]:
detections = vision_module.get_detections()
print(f"✅ Test 2: Detecciones obtenidas ({len(detections)} objetos detectados)")
if(len(detections))> 0:
    print(detections[0]['bbox'])
assert isinstance(detections, list), "❌ Las detecciones no son una lista"
for d in detections:
    assert 'bbox' in d and 'score' in d and 'class_id' in d, "❌ Formato de detección inválido"


<h2>Test 3. Validación Lectura Sensor Proximidad</h2>

In [ ]:
distance = vision_module.get_nearest_object_distance()
print(f"✅ Test 3: Distancia detectada: {distance} cm")
assert isinstance(distance, (float, int)), "❌ La distancia no es un número"


<h2>Test 4. Validación Movimiento Coche</h2>

In [ ]:
car = CarSim(client_id)

print("Moviendo hacia adelante...")
car.move_forward(3)

print("Parado...")
car.stop()

print("Girando a la izquierda...")
car.turn_left(5)

print("Parado...")
car.stop()

print("Girando a la derecha...")
car.turn_right(10)

print("Parado...")
car.stop()

print("✅ Test 4: Movimiento completado")


<h2>Test 5. Validación Decisión Coche</h2>

In [ ]:
if len(detections) > 0:
    print("✅ Objeto detectado con IA")
elif 0 < distance < 1.5:
    print("⚠️ Posible pared detectada por sensor")
else:
    print("ℹ️ Nada detectado")


<h2>Test 6. Validación Buscar Objeto</h2>

In [ ]:
detections2 = []
speed = 2
radio_rueda = 0.05 # en coppelia
while len(detections2) == 0:
    detections2 = vision_module.get_detections()
    print(f"Detecciones obtenidas ({len(detections2)} objetos detectados)")
    if len(detections2) > 0:
        break

    segundos = calcular_tiempo_rotacion(90, 2, 0.03)
    print(segundos)
    car.turn_right(1)
    time.sleep(2)

distance = vision_module.get_nearest_object_distance()
print(f"Distancia detectada: {distance} cm")
res, resolution, image = sim.simxGetVisionSensorImage(client_id, vision_module.sensor_handle, 0, sim.simx_opmode_oneshot_wait)
img = np.array(image, dtype=np.float32)
img = (1.0 - img) * 255          
img = img.astype(np.uint8)
img = img.reshape((resolution[1], resolution[0], 3))
img = cv2.flip(img, 0)
bug2(client_id, vision_module, car, img, detections2[0])
    

<h1>Test 7: Agafar un objecte</h1>

In [ ]:
pinza = Pinca(client_id, vision_module)
pinza.agafar_objecte()